In [1]:
import time

import requests

from config import LAST_FM_API_KEY, LAST_FM_USER_AGENT
from LastFM_scraping import get_tag_artists

In [2]:
url = 'http://ws.audioscrobbler.com/2.0/'
headers = {
    'user-agent': LAST_FM_USER_AGENT
}
lastFM_tags = [
    'polish hip hop',
    'polski hip hop',
    'polish hip-hop',
    'polski hip-hop',
    'polski rap',
    'polish rap']

In [21]:
tag_artists_dict = {}
for tag in lastFM_tags:
    artists_with_tag = get_tag_artists(tag, max_pages=200)
    tag_artists_dict[tag] = artists_with_tag

Reached max page
Number of pages searched 6
Searching page nr 10
Searching page nr 20
Reached max page
Number of pages searched 24
Searching page nr 10
Searching page nr 20
Reached max page
Number of pages searched 24
Searching page nr 10
Reached max page
Number of pages searched 12
Searching page nr 10
Reached max page
Number of pages searched 15
Searching page nr 10
Reached max page
Number of pages searched 17


Looking at the @attr of the response:  ({'page': '201', 'perPage': '50', 'tag': 'polish', 'total': '18964', 'totalPages': '380'})
Total number of polish artists from the response is 18964 which coresponds with the total pages (380) but i recieve only the most popular 10k artists. That is also what happens when we go to popular tags on lastFM it only shows 10k artists (https://www.last.fm/tag/french/artists?page=477).

In [15]:
polish_artists = get_tag_artists('polish', max_pages=200)
polish_artists

Searching page nr 10
Searching page nr 20
Searching page nr 30
Searching page nr 40
Searching page nr 50
Searching page nr 60
Searching page nr 70
Searching page nr 80
Searching page nr 90
Searching page nr 100
Searching page nr 110
Searching page nr 120
Searching page nr 130
Searching page nr 140
Searching page nr 150
Searching page nr 160
Searching page nr 170
Searching page nr 180
Searching page nr 190
Searching page nr 200


[Artysta(name='Myslovitz', url='https://www.last.fm/music/Myslovitz'),
 Artysta(name='Sanah', url='https://www.last.fm/music/Sanah'),
 Artysta(name='Brodka', url='https://www.last.fm/music/Brodka'),
 Artysta(name='WHITE 2115', url='https://www.last.fm/music/WHITE+2115'),
 Artysta(name='Chivas', url='https://www.last.fm/music/Chivas'),
 Artysta(name='Jacek Kaczmarski', url='https://www.last.fm/music/Jacek+Kaczmarski'),
 Artysta(name='Maria Peszek', url='https://www.last.fm/music/Maria+Peszek'),
 Artysta(name='Nosowska', url='https://www.last.fm/music/Nosowska'),
 Artysta(name='Kazik', url='https://www.last.fm/music/Kazik'),
 Artysta(name='Daria Zawiałow', url='https://www.last.fm/music/Daria+Zawia%C5%82ow'),
 Artysta(name='Cool Kids Of Death', url='https://www.last.fm/music/Cool+Kids+Of+Death'),
 Artysta(name='Kizo', url='https://www.last.fm/music/Kizo'),
 Artysta(name='Marek Grechuta', url='https://www.last.fm/music/Marek+Grechuta'),
 Artysta(name='Mela Koteluk', url='https://www.last.

In [53]:
def tags_in_artist_tags(artist_name, tag_names):
    # checks if any of the provided tags are present in the LastFM list of tags of a artist
    # sleep to appease the API gods (I don't know if i need it)
    time.sleep(0.1)
    params = {
        'artist': artist_name,
        'api_key': LAST_FM_API_KEY,
        'method': 'artist.getTopTags',
        'format': 'json'
    }
    # this allows for using just one tag name
    if type(tag_names) == str:
        tag_names = [tag_names]
    tag_names = [tag_name.lower() for tag_name in tag_names]
    data = requests.get(url=url, params=params, headers=headers).json()
    for tag in data['toptags']['tag']:
        if tag['name'].lower() in tag_names:
            return True
    return False


In [54]:
tags_in_artist_tags('Paktofonika', 'rap')

True

Takes really long to execute, don't really have good idea how to check it in a different way

In [40]:
rap_or_hiphop_in_tags = [tags_in_artist_tags(a.name, ['rap', 'hip-hop', 'hip hop']) for a in polish_artists]
polish_rappers = [artist for artist, is_a_rapper, in zip(polish_artists, rap_or_hiphop_in_tags) if is_a_rapper]
tag_artists_dict['polish_and_hiphop'] = polish_rappers
len(polish_rappers)

1413

this is probably unnecessary
from dataclasses import dataclass
@dataclass
class Artysta:
    name: str
    url: str
    def __key(self):
         return self.name
    def __hash__(self):
        return hash(self.__key())
    def __eq__(self, other):
        if isinstance(other, Artysta):
            return self.__key() == other.__key()
        if isinstance(other, str):
            return self.name == other
        return False

In [23]:
from functools import reduce
def reduce_tables(tables):
    #make them into sets
    tables = [set(table) for table in tables]
    # union of sets
    # this can probably be done with set().union(tables) or sth
    return reduce(lambda s1, s2: s1 | s2, tables)
reduced = reduce_tables(tag_artists_dict.values())

{Artysta(name='FFOD', url='https://www.last.fm/music/FFOD'),
 Artysta(name='Rapanui', url='https://www.last.fm/music/Rapanui'),
 Artysta(name='Dj Piekarz', url='https://www.last.fm/music/Dj+Piekarz'),
 Artysta(name='Zaginiony', url='https://www.last.fm/music/Zaginiony'),
 Artysta(name='Sobota feat. Kool Savas, Gural, Wall-e, Rytmus, Bigz, Matheo', url='https://www.last.fm/music/Sobota+feat.+Kool+Savas,+Gural,+Wall-e,+Rytmus,+Bigz,+Matheo'),
 Artysta(name='B.O.K (BiszOerKay/Dj Paulo)', url='https://www.last.fm/music/B.O.K+(BiszOerKay%2FDj+Paulo)'),
 Artysta(name='Małolat/Ajron', url='https://www.last.fm/music/Ma%C5%82olat%2FAjron'),
 Artysta(name='piotr wroczyński', url='https://www.last.fm/music/piotr+wroczy%C5%84ski'),
 Artysta(name='Roka', url='https://www.last.fm/music/Roka'),
 Artysta(name='Bonus RPK', url='https://www.last.fm/music/Bonus+RPK'),
 Artysta(name='FSOUNDS', url='https://www.last.fm/music/FSOUNDS'),
 Artysta(name='WRR', url='https://www.last.fm/music/WRR'),
 Artysta(nam

In [24]:
import pickle

with open('../scraped_data/LastFm/' + 'polish_rappers_LastFM.pkl', 'rb') as f:
    pl_rappers = pickle.load(f)
print(f"nr of rappers with tags 'polish' and 'hip hop': {len(pl_rappers)}")
print(f"nr of rappers with any of the other tags (polski rap etc.) {len(reduced)}")
all_pl_rappers = reduce_tables([pl_rappers, reduced])
print(f"nr of rappers combined {len(all_pl_rappers)}")


nr of rappers with tags 'polish' and 'hip hop': 1413
nr of rappers with any of the other tags (polski rap etc.) 2706
nr of rappers combined 3623


Cleaning from features

In [58]:
def is_a_feature(name):
    split_name = name.lower().split()
    # maybe this should be done by the regex search?
    features_words = [
        'feat',
        'feat.',
        '(feat.',
        '(feat',
        'ft',
        'ft.',
        '(ft.',
        '(ft'
    ]
    for word in split_name:
        if word in features_words:
            return True
    return False

no_features = [a for a in all_pl_rappers if not is_a_feature(a.name)]

with open('../scraped_data/LastFm/' + 'all_no_features_LastFM.pkl', 'wb') as f:
    pickle.dump(no_features, f)
with open('../scraped_data/LastFm/' + 'all_no_features_LastFM.txt', 'w', encoding='utf-8') as f:
    for a in no_features:
        f.write(a.name + '\n')

In [26]:
names_all = [a.name for a in all_pl_rappers]
'Lona i Webber' in names_all

True

In [39]:
import pickle

with open('../scraped_data/LastFm/' + 'all_polish_rappers_LastFM.pkl', 'wb') as f:
    pickle.dump(all_pl_rappers, f)

In [40]:
with open('../scraped_data/LastFm/' + 'all_polish_rappers_LastFM.txt', 'w', encoding='utf-8') as f:
    for a in all_pl_rappers:
        f.write(a.name + '\n')

In [43]:
with open('../scraped_data/LastFM_artists.pkl', 'rb') as f:
    polish_hiphop_tag_lastfm = pickle.load(f)

In [48]:
names_of_polish_rappers = [a.name for a in polish_rappers]

In [50]:
len(list(set(polish_hiphop_tag_lastfm) & set(names_of_polish_rappers)))

48

In [12]:
def save_tag_artists_txt(tag_name, l_artists):
    scraped_data_path = '../scraped_data/'
    with open(scraped_data_path + tag_name + ' LastFM_artist.txt', 'w', encoding="utf-8") as txtfile:
        for artist in l_artists:
            txtfile.write(artist.name + ',' + artist.url + ',\n')


save_tag_artists_txt('polish', polish_artists)

In [7]:
hiphop_artists = get_tag_artists('hip hop', max)

Error: Invalid API key - You must be granted a valid key by last.fm


The number of artists seems to be the same when we group by name in lowercase

In [8]:
len(artists) == len(set(map(lambda x: x.lower(), artists)))

AttributeError: 'Artysta' object has no attribute 'lower'

Taking a look at the names of artist that have features in them and filtering them out

In [ ]:
features = list(filter(lambda x: 'feat' in x, artists))
artists = list(filter(lambda x: 'feat' not in x, artists))
print('filtered out')
print(features)

In [ ]:
list(filter(lambda x: '\"' in x, artists))

In [ ]:
import pickle

scraped_data_path = '../scraped_data/'
with open(scraped_data_path + 'LastFM_artists.pkl', 'wb') as csvfile:
    pickle.dump(artists, csvfile)
with open(scraped_data_path + 'LastFM_artists.pkl', 'rb') as f:
    loaded_artist = pickle.load(f)
loaded_artist


In [ ]:
with open(scraped_data_path + tag + 'LastFM_artist.txt', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        print(row)

In [ ]:
for row in reader:
    print(row)